In [1]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce GTX 1650
Using device: cuda


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Define Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 64 * 64 * 3),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z).view(z.size(0), 3, 64, 64)

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 64 * 3, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.model(img)

# Training function
def train_gan(generator, discriminator, data_loader, latent_dim, epochs, device, save_path):
    criterion = nn.BCELoss()
    optimizer_g = optim.Adam(generator.parameters(), lr=0.0002)
    optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002)

    os.makedirs(save_path, exist_ok=True)  # Ensure the save directory exists

    for epoch in range(epochs):
        for real_imgs, _ in data_loader:
            real_imgs = real_imgs.to(device).view(real_imgs.size(0), -1)

            # Train Discriminator
            valid = torch.ones(real_imgs.size(0), 1).to(device)
            fake = torch.zeros(real_imgs.size(0), 1).to(device)

            optimizer_d.zero_grad()
            real_loss = criterion(discriminator(real_imgs), valid)

            z = torch.randn(real_imgs.size(0), latent_dim).to(device)
            fake_imgs = generator(z)
            fake_loss = criterion(discriminator(fake_imgs.view(fake_imgs.size(0), -1)), fake)

            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train Generator
            optimizer_g.zero_grad()
            z = torch.randn(real_imgs.size(0), latent_dim).to(device)
            fake_imgs = generator(z)
            g_loss = criterion(discriminator(fake_imgs.view(fake_imgs.size(0), -1)), valid)

            g_loss.backward()
            optimizer_g.step()

        print(f"Epoch [{epoch + 1}/{epochs}] - D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

        # Save models periodically
        if (epoch + 1) % 5 == 0 or epoch == epochs - 1:  # Save every 5 epochs and the final epoch
            torch.save(generator.state_dict(), os.path.join(save_path, "generator.pth"))
            torch.save(discriminator.state_dict(), os.path.join(save_path, "discriminator.pth"))
            print(f"Models saved at epoch {epoch + 1}")

# Main script
if __name__ == "__main__":
    latent_dim = 100
    batch_size = 64
    epochs = 100
    save_path = "./models"  # Directory to save models
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Dataset
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    dataset = datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Models
    generator = Generator(latent_dim).to(device)
    discriminator = Discriminator().to(device)

    # Train
    train_gan(generator, discriminator, data_loader, latent_dim, epochs, device, save_path)


100%|██████████| 170498071/170498071 [00:26<00:00, 6543405.76it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Epoch [1/100] - D Loss: 0.2679, G Loss: 4.8674
Epoch [2/100] - D Loss: 0.0970, G Loss: 2.7648
Epoch [3/100] - D Loss: 0.3418, G Loss: 3.5432
Epoch [4/100] - D Loss: 0.4498, G Loss: 2.5483
Epoch [5/100] - D Loss: 1.7149, G Loss: 4.5562
Models saved at epoch 5
Epoch [6/100] - D Loss: 0.8848, G Loss: 2.5202
Epoch [7/100] - D Loss: 0.7654, G Loss: 3.0090
Epoch [8/100] - D Loss: 1.4406, G Loss: 1.7960
Epoch [9/100] - D Loss: 1.0109, G Loss: 1.6095
Epoch [10/100] - D Loss: 0.6393, G Loss: 1.6690
Models saved at epoch 10
Epoch [11/100] - D Loss: 0.8843, G Loss: 1.7096
Epoch [12/100] - D Loss: 0.9642, G Loss: 1.4773
Epoch [13/100] - D Loss: 0.9911, G Loss: 2.9570
Epoch [14/100] - D Loss: 0.9604, G Loss: 1.5489
Epoch [15/100] - D Loss: 1.0213, G Loss: 1.1879
Models saved at epoch 15
Epoch [16/100] - D Loss: 1.4011, G Loss: 1.4107
Epoch [17/100] - D Loss: 1.1780, G Loss: 1.3822
Epoch [18/100] - D Loss: 1.2817, G Loss: 1.7663
Epoch [19/100] - D L